In [1]:
import pandas as pd
import pickle
import xgboost as xgb
import nfl_data_py as nfl

In [2]:
overUnderTenModel = pickle.load(open('overUnder10Model.pkl', 'rb'))
overUnderTwelveModel = pickle.load(open('overUnder12Model.pkl', 'rb'))
overUnderFourteenModel = pickle.load(open('overUnder14Model.pkl', 'rb'))
overUnderSixteenModel = pickle.load(open('overUnder16Model.pkl', 'rb'))
overUnderEighteenModel = pickle.load(open('overUnder18Model.pkl', 'rb'))
overUnderTwentyModel = pickle.load(open('overUnder20Model.pkl', 'rb'))

In [3]:
rbData = pd.read_parquet("rbDataV2.parquet")
wrData = pd.read_parquet("wrDataV2.parquet")
teData = pd.read_parquet("teDataV2.parquet")

rbData['position'] = [0 for i in range(rbData.shape[0])]
wrData['position'] = [1 for i in range(wrData.shape[0])]
teData['position'] = [2 for i in range(teData.shape[0])]

df = pd.concat([rbData, wrData, teData])

In [4]:
curData = df[df.season == 2021].copy()

In [5]:
colsToDrop = ['player_id', 'season',
              'fantasy_points_ppr_future', 
              'fantasy_points_ppr_per_game_future',
              'fantasy_rank_future',
              'per_game_fantasy_rank_future']
# separate featues and labels
curFeatures = curData.drop(colsToDrop, axis=1)

In [6]:
curData['over_ten'] = overUnderTenModel.predict(curFeatures)
curData['over_twelve'] = overUnderTwelveModel.predict(curFeatures)
curData['over_fourteen'] = overUnderFourteenModel.predict(curFeatures)
curData['over_sixteen'] = overUnderSixteenModel.predict(curFeatures)
curData['over_eighteen'] = overUnderEighteenModel.predict(curFeatures)
curData['over_twenty'] = overUnderTwentyModel.predict(curFeatures)

In [7]:
curData.head()

,player_id,season,age,height,weight,experience,draft_pick,games,carries,rushing_yards,...,fantasy_points_ppr_per_game_n-2,fantasy_rank_n-2,per_game_fantasy_rank_n-2,position,over_ten,over_twelve,over_fourteen,over_sixteen,over_eighteen,over_twenty
0,00-0032764,2021,27,75.0,247.0,5.0,14.0,8,219,937.0,...,19.640000,5.0,4.0,0,1,1,1,1,0,0
1,00-0036223,2021,22,71.0,221.0,1.0,9.0,17,332,1811.0,...,NaN,NaN,NaN,0,1,1,1,1,0,0
2,00-0033699,2021,26,70.0,200.0,4.0,NaN,16,206,911.0,...,19.312500,4.0,6.0,0,1,1,1,0,0,0
3,00-0033856,2021,26,72.0,228.0,4.0,4.0,14,180,812.0,...,17.293333,7.0,9.0,0,1,1,0,0,0,0
4,00-0033280,2021,25,71.0,205.0,4.0,8.0,7,99,442.0,...,29.450000,1.0,1.0,0,1,1,1,0,0,0


In [8]:
idData = nfl.import_ids()
idData = idData[['gsis_id', 'name', 'team']]
idData.head()

,gsis_id,name,team
0,NaN,Matt Corral,CAR
1,NaN,Kenny Pickett,PIT
2,NaN,Sam Howell,WAS
3,NaN,Malik Willis,TEN
4,NaN,Carson Strong,PHI


In [9]:
mergedData = curData.merge(idData, left_on='player_id', right_on='gsis_id')

In [10]:
prediction = []
for row in mergedData.itertuples():
    if row.over_twenty == 1:
        prediction.append('20<')
    elif row.over_eighteen == 1:
        prediction.append('18-20')
    elif row.over_sixteen == 1:
        prediction.append('16-18')
    elif row.over_fourteen == 1:
        prediction.append('14-16')
    elif row.over_twelve == 1:
        prediction.append('12-14')
    elif row.over_ten == 1:
        prediction.append('10-12')
    else:
        prediction.append('<10')

In [11]:
mergedData.columns

Index(['player_id', 'season', 'age', 'height', 'weight', 'experience',
       'draft_pick', 'games', 'carries', 'rushing_yards',
       ...
       'position', 'over_ten', 'over_twelve', 'over_fourteen', 'over_sixteen',
       'over_eighteen', 'over_twenty', 'gsis_id', 'name', 'team'],
      dtype='object', length=174)

In [12]:
mergedData['prediction'] = prediction

In [13]:
subset = mergedData[['gsis_id', 'name', 'position', 'team', 'prediction', 'age', 'fantasy_points_ppr_per_game', 'fantasy_rank']].copy()

In [14]:
updatedPositionsDict = {0: 'RB', 1: 'WR', 2: 'TE'}
updatedPositions = []
for row in subset.itertuples():
    updatedPositions.append(updatedPositionsDict[row.position])

In [15]:
subset['position'] = updatedPositions

In [16]:
subset.head(35)

,gsis_id,name,position,team,prediction,age,fantasy_points_ppr_per_game,fantasy_rank
0,00-0032764,Derrick Henry,RB,TEN,16-18,27,24.162500,22
1,00-0036223,Jonathan Taylor,RB,IND,16-18,22,21.947059,1
2,00-0033699,Austin Ekeler,RB,LAC,14-16,26,21.487500,2
3,00-0033856,Leonard Fournette,RB,TBB,12-14,26,18.257143,6
4,00-0033280,Christian McCaffrey,RB,CAR,14-16,25,18.214286,38
5,00-0033906,Alvin Kamara,RB,NOS,14-16,26,18.053846,8
6,00-0033897,Joe Mixon,RB,CIN,14-16,25,17.993750,4
7,00-0036893,Najee Harris,RB,PIT,14-16,23,17.688235,3
8,00-0033553,James Conner,RB,ARI,12-14,26,17.180000,5
9,00-0036275,D'Andre Swift,RB,DET,12-14,22,16.069231,15


In [17]:
subset.to_parquet('flex_predictions.parquet')